# Graduated Non-Convexity (GNC) for robust spatial perceptions

The next big thing following RANSAC

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from numpy.random import default_rng
rng = default_rng()

mean = [0, 0]
cov = [[1,0.99], [0.99, 1]]  # diagonal covariance

vals = rng.multivariate_normal(mean,cov,100)

mean2 = [0, 0]
cov2 = [[2,0], [0, 2]]  # diagonal covariance

vals2 = rng.multivariate_normal(mean2,cov2,100)

vals3 = np.vstack((vals,vals2))

model = np.polyfit(vals3[:,0],vals3[:,1],1)   
predict = np.poly1d(model)

x_lin_reg = range(-4, 4)
y_lin_reg = predict(x_lin_reg)

plt.plot(x_lin_reg, y_lin_reg, c = 'r')

plt.scatter(vals[:,0],vals[:,1])
plt.scatter(vals2[:,0],vals2[:,1])

plt.axis('equal')
plt.show()